In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [2]:
train = pd.read_csv('train_modelling_new_inf.csv')
train

,full_sq,num_room,state,culture_objects_top_25,build_count_monolith,ttk_km,railroad_km,cafe_count_500_na_price,cafe_count_1000_price_high,trc_count_1500,...,max_floor_ratio,cpi,gdp_annual_growth,usdrub,invest_fixed_capital_per_cap,utility_sq,life_sq_ratio,room_size,rel_floor,price_doc
0,43.0,2,2.07165,0,2,10.918587,1.305159,0,0,9,...,1.000000,354.0,0.045037,29.0048,73976.19863,16,0.627907,18.535184,0.588235,5850000
1,34.0,2,2.07165,1,4,3.103996,0.694536,0,0,7,...,1.000000,354.0,0.045037,28.9525,73976.19863,15,0.558824,18.535184,0.588235,6000000
2,43.0,2,2.07165,0,4,2.927487,0.700691,0,0,1,...,1.000000,354.0,0.045037,28.8082,73976.19863,14,0.674419,18.535184,0.588235,5700000
3,89.0,2,2.07165,0,50,14.606501,1.999265,0,0,7,...,1.000000,353.2,0.045037,28.9655,73976.19863,39,0.561798,18.535184,0.588235,13100000
4,77.0,2,2.07165,0,16,1.721834,0.084113,3,1,9,...,1.000000,353.2,0.045037,29.4625,73976.19863,0,1.000000,18.535184,0.588235,16331452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,44.0,2,3.00000,0,5,8.361875,1.137603,1,0,6,...,0.750000,489.5,0.007065,55.2655,131403.00000,17,0.613636,13.500000,0.777778,7400000
30467,86.0,4,3.00000,1,25,1.238732,0.377368,3,1,7,...,0.529412,489.5,0.007065,55.2655,131403.00000,27,0.686047,14.750000,0.333333,25000000
30468,45.0,1,1.00000,0,6,17.148737,1.727223,0,0,0,...,1.176471,489.5,0.007065,55.2655,131403.00000,18,0.596154,18.535184,0.500000,6970959
30469,64.0,2,2.00000,0,9,8.940313,4.898047,0,0,1,...,0.882353,489.5,0.007065,55.2655,131403.00000,32,0.500000,16.000000,0.333333,13500000


In [3]:
test = pd.read_csv('test_modelling_new_inf.csv')
test

,full_sq,num_room,state,culture_objects_top_25,build_count_monolith,ttk_km,railroad_km,cafe_count_500_na_price,cafe_count_1000_price_high,trc_count_1500,...,median_max_floor,max_floor_ratio,cpi,gdp_annual_growth,usdrub,invest_fixed_capital_per_cap,utility_sq,life_sq_ratio,room_size,rel_floor
0,39.00,1,3.00000,0,116,19.651101,0.490549,0,0,0,...,14.0,0.642857,490.5,0.007065,55.5989,131403,18.30,0.530769,20.700000,0.222222
1,79.20,3,1.00000,0,6,16.975793,2.342346,0,0,0,...,14.5,1.172414,490.5,0.007065,55.5989,131403,18.00,0.596154,18.535184,0.470588
2,40.50,2,2.00000,0,3,5.627481,2.220941,1,0,1,...,5.0,1.000000,490.5,0.007065,55.5989,131403,15.40,0.619753,12.550000,0.600000
3,62.80,2,3.00000,0,6,22.094252,4.476081,0,0,0,...,10.0,1.700000,490.5,0.007065,55.5989,131403,26.80,0.573248,18.000000,1.000000
4,40.00,1,1.00000,0,6,17.148737,1.727223,0,0,0,...,14.5,1.172414,490.5,0.007065,55.5989,131403,0.00,1.000000,40.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,52.20,2,2.00000,0,3,7.927396,5.337760,0,0,9,...,12.0,1.000000,523.2,-0.037267,65.5700,126874,20.40,0.609195,15.900000,0.833333
7658,54.09,2,2.07165,0,6,24.382134,8.607590,0,0,0,...,17.0,0.000000,523.2,-0.037267,65.9385,126874,18.00,0.596154,18.535184,0.000000
7659,41.08,1,1.00000,1,25,4.081283,2.566044,9,1,25,...,11.0,0.090909,523.2,-0.037267,65.6745,126874,40.08,0.024343,1.000000,12.000000
7660,34.80,1,2.00000,0,3,11.538742,2.461916,0,0,6,...,12.0,0.750000,523.2,-0.037267,65.6745,126874,15.00,0.568966,19.800000,0.888889


In [4]:
df = pd.read_csv('submission.csv')
df

,id,price_doc
0,30474,7118500.41
1,30475,7118500.41
2,30476,7118500.41
3,30477,7118500.41
4,30478,7118500.41
...,...,...
7657,38131,7118500.41
7658,38132,7118500.41
7659,38133,7118500.41
7660,38134,7118500.41


In [5]:
np.isinf(test).values.sum()

0

In [6]:
for column in train.columns.values:
    if abs(train[column].corr(train['price_doc'])) < 0.1:
        train = train.drop(column, axis=1)
        if column in test.columns.values:
            test = test.drop(column, axis=1)

In [7]:
for column in train.columns.values:
    if stats.skew(train[column].values) > 1:
        train[column] = np.log(train[column] + 1)   
        if column in test.columns.values:
            test[column]  = np.log(test[column] + 1)   

In [8]:
X = train.drop("price_doc", axis=1)
y = train["price_doc"]

In [9]:
scaler = StandardScaler();
scaler.fit(X);
scaled_train = scaler.transform(X);

test_scaler = StandardScaler();
test_scaler.fit(test);
scaled_test = test_scaler.transform(test);

## Linear Regression

In [10]:
lr = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=-1)

In [11]:
lr.fit(scaled_train, y)

C:\Users\pongp\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


LinearRegression(n_jobs=-1, normalize=True)

In [12]:
y_pred = np.maximum(np.exp(lr.predict(scaled_test))-1, 0)

In [13]:
df['price_doc'] = y_pred
df

,id,price_doc
0,30474,4.105772e+06
1,30475,8.236450e+06
2,30476,5.004139e+06
3,30477,4.929688e+06
4,30478,4.394834e+06
...,...,...
7657,38131,6.800457e+06
7658,38132,4.122353e+06
7659,38133,4.118286e+06
7660,38134,4.187929e+06


In [14]:
df.to_csv("Linear_new.csv", index=False)

## Ridge

In [ ]:
ridge = Ridge(alpha=60, copy_X=False, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
ridge.fit(X, y)

In [ ]:
y_pred = ridge.predict(scaled_test)

In [ ]:
df['price_doc'] = y_pred
df

## Lasso

In [ ]:
lasso = Lasso(alpha=0.001)

In [ ]:
lasso.fit(X_train, y_train);

In [ ]:
y_pred = ridge.predict(X_test)

In [ ]:
print("MSE: ", mean_squared_error(y_test, y_pred))
print("RMSE: ", mean_squared_error(y_test, y_pred)**0.5)

In [ ]:
x_ax = range(len(y_test))
plt.figure(figsize=(12, 6))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, y_pred, label="predicted")
plt.title("Russian Housing dataset test and predicted data")
plt.xlabel('X')
plt.ylabel('Price')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()